---
title: Transfer graph into hypergraph
subtitle: Expansion 기법별 Hyper-graph 에서의 Graph structure로의 전환 방법과 그 과정에서의 정보가 손실률 추적
description: blank
categories:
  - HAR/benchmarks
author: YeEun Hong
date: 2023-05-03
---

> 본 포스트에서 사용하는 framework DHG에 대한 자세한 설명은 포스트를 참고: [DHG](https://4923.github.io/archive/HAR/frameworks/dhg.html)

## 개요

- 가설
    - 그래프는 {} 이고 하이퍼 그래프는 {} 이다. 1:1 vertex structure를 가지는 그래프 구조로 인해 관절 간 다층적 연결관계를 확인하기 어려운 현재 handcraft skeleton structure를 HyperGraph로 재정의하여 인체공학적 성질을 더한다.
    - Fully connected Graph인 HD-GCN보다는 더 적은 FLOPS를, 기존 ST-GCN++ 또는 현재 SOTA에 준하는 성능을 보일 것이다.
- 방향성
    - 그래프는 하이퍼그래프의 특수한 경우로, 많은 HAR 기법이 Graph Based Model을 차용하고 있으므로 인체 구조를 HyperGraph로 정의한 후 Graph로 변환한다.
    - 이 때 정보가 손실을 확인하고, 모델에 입력값으로 넣을 방법을 찾는다.
    
- 대안적 방향: Graph를 HyperGraph로 변환하여 HyperGraph 모델인 `HGCN`, `HGNN`, `HGNN++` 에 입력하는 방법을 찾는다.
    > 위 방향은 새로운 노트북을 통해 실험할 것.
    - HyperNeuralNetwork는 학습할 수 있는 데이터의 형태가 정해져 있으므로 데이터셋을 변환하는 방법에 대해 확인하고 찾아보아야 한다.
    - 이 때에도 단순 NTU style to Cora style 변형이 아닌 관절 간 다층적 연결관계를 반영해야 할 것이다.


### 실험 방법
1. Expansion
    1. HyperGraph handling framework `dhg` 를 사용하여 임의의 HyperGraph를 생성하고 Incidence Matrix를 정의한다.
    2. expansion 기법은 총 세가지인데, 각 방법에 따라 HyperGraph를 Graph 구조로 변형하며 Adjacency Matrix로 변형한다.
    3. 이 때 결과가 되는 Adjacency Matrix의 형태가 어떻게 변화하는지 추적한다.
2. 각 expansion 별 정보가 손실률을 측정한다. 이 때 정보가를 정의하는 방법 또한 정의해야한다.

### 실험 목표
1. 임의의 Incidence Matrix를 가지는 Hyper-graph structure $H$를 dhg에서 제공하는 expansion 기법을 사용하여 Graph $G$로 변환했을 때 어떤 Adjacency Matrix가 나오는지 확인한다.
2. $H$가 $G$로 변환할 때 $H$ 의 정보가를 95% 이상 유지하는 expansion 방법을 찾는다.
3. 각각을 baseline으로 삼은 `DG-STGCN`에 입력값으로 넣어 학습 결과를 비교한다.

### 실험 과정

- 모든 sample code는 [dhg의 expansion example](https://deephypergraph.readthedocs.io/en/latest/tutorial/structure.html#reduced-from-high-order-structures) 을 따른다.

In [3]:
"""
Base settings
---
Conda Environment: hypergraph-py38
Python version: python 3.8.16
Requirements: dhg, tqdm
"""

import dhg, tqdm

# for function annotation
from typing import Tuple, List

/opt/homebrew/Caskroom/miniforge/base/envs/hypergraph-py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# https://deephypergraph.readthedocs.io/en/latest/api/dhg.html?highlight=dhg.hypergraph#dhg.Hypergraph

# 1. hyper graph declaration
# class dhg.Hypergraph(num_v, e_list=None, e_weight=None, v_weight=None, merge_op='mean', device=torch.device)

hg = dhg.Hypergraph(5, [(0, 1, 2), (1, 3, 2), (1, 2), (0, 3, 4)])


# 2. print hyper graph edges
# property D_e: Return the hyperedge degree matrix with torch.sparse_coo_tensor format. (torch.Tensor)

hg.e    # Tuple[List[List[int]], List[float]] -> ([hyper graph edges], [its weights])

([(0, 1, 2), (1, 2, 3), (1, 2), (0, 3, 4)], [1.0, 1.0, 1.0, 1.0])

In [5]:
# 3. print hypergraph incidence matrix ($H$)
# Return the hypergraph incidence matrix with torch.sparse_coo_tensor format. (torch.Tensor)

hg.H.to_dense()    # Tensor

tensor([[1., 0., 0., 1.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [0., 1., 0., 1.],
        [0., 0., 0., 1.]])

`hg.W_e` [docs](https://deephypergraph.readthedocs.io/en/latest/api/dhg.html?highlight=dhg.hypergraph#dhg.Hypergraph.W_e) 에서 incidences matrix의 weights와 실제 values로 들어간 값 (hg.e로 입력한)을 확인할 수 있다. indices가 왜 torch.Size([2,4]) 인지는 잘 모르겠다...

```py
hg.W_e
>> tensor(indices=tensor([[0, 1, 2, 3],
                       [0, 1, 2, 3]]),
       values=tensor([1., 1., 1., 1.]),
       size=(4, 4), nnz=4, layout=torch.sparse_coo)
```

In [6]:
# Size check

import torch

def check_tensor_size(v:int, edges:List[Tuple[int]], weights) -> None:
    hg = dhg.Hypergraph(v, edges, weights)
    print("==="*10)
    print("Incidence Matrix of H")
    print(hg.H.to_dense())
    print("---"*10)
    print("hg.W_e : ", hg.W_e)
    print("Tensor Size : ", hg.H.to_dense().size() )    # origin H size
    print("Tensor Size : ", hg.W_e.indices().size() )      # indices size
    print("==="*10)


check_tensor_size(5, [(0, 1, 2), (1, 3, 2), (1, 2), (0, 3, 4)], [1, 1, 1, 1])
check_tensor_size(5, [(0, 1, 2), (1, 3, 2), (1, 2), (0, 3, 4)], [2, 2, 1, 1])
# check_tensor_size(5, [(5,9,21), (3,6,21), (3,10,21)], [1, 1, 1])

# > 별 차이가 없는데 W_e 는 이렇게 쓰는게 아닌것 같음.
# > 3개로 고정해서 각 3개짜리의 그래프의 가중치를 찾고 그걸 가지고 뭘 해도...

Incidence Matrix of H
tensor([[1., 0., 0., 1.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [0., 1., 0., 1.],
        [0., 0., 0., 1.]])
------------------------------
hg.W_e :  tensor(indices=tensor([[0, 1, 2, 3],
                       [0, 1, 2, 3]]),
       values=tensor([1., 1., 1., 1.]),
       size=(4, 4), nnz=4, layout=torch.sparse_coo)
Tensor Size :  torch.Size([5, 4])
Tensor Size :  torch.Size([2, 4])
Incidence Matrix of H
tensor([[1., 0., 0., 1.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [0., 1., 0., 1.],
        [0., 0., 0., 1.]])
------------------------------
hg.W_e :  tensor(indices=tensor([[0, 1, 2, 3],
                       [0, 1, 2, 3]]),
       values=tensor([2., 2., 1., 1.]),
       size=(4, 4), nnz=4, layout=torch.sparse_coo)
Tensor Size :  torch.Size([5, 4])
Tensor Size :  torch.Size([2, 4])


##### Star Expansion

> reat the hyperedges in the hypergraph as virtual vertices in the graph

In [7]:
# v_mask (virtual vertex mask) 는 실제 정점여부를 나타내는 vertex mask로 True (1) 이면 actual vertex, False (0) 이면 virtual vertex다.

g, v_mask = dhg.Graph.from_hypergraph_star(hg)

In [10]:
print(f"hyper graph information : {hg}")
print(f"graph information : {g}")

hyper graph information : Hypergraph(num_v=5, num_e=4)
graph information : Graph(num_v=9, num_e=11)


In [16]:
# g.e 는 동일하게 [0]에서 연결된 edge list를, [1]에서 각각의 weight list를 반환한다.
print(f"Converted Graph edge list : " , g.e[0])

Converted Graph edge list :  [(0, 5), (0, 8), (1, 5), (1, 6), (1, 7), (2, 5), (2, 6), (2, 7), (3, 6), (3, 8), (4, 8)]


In [24]:
print(f"v_mask(Tensor) : {v_mask}")
print(f"shape or size of v_mask : {v_mask.size()}")
print(f"\nCompared v_mask.size() to number of vertex : {v_mask.size()} == {g.v}")

v_mask(Tensor) : tensor([ True,  True,  True,  True,  True, False, False, False, False])
shape or size of v_mask : torch.Size([9])

Compared v_mask.size() to number of vertex : torch.Size([9]) == [0, 1, 2, 3, 4, 5, 6, 7, 8]


In [26]:
g.A.to_dense(), g.A.to_dense().size()

(tensor([[0., 0., 0., 0., 0., 1., 0., 0., 1.],
         [0., 0., 0., 0., 0., 1., 1., 1., 0.],
         [0., 0., 0., 0., 0., 1., 1., 1., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [1., 1., 1., 0., 0., 0., 0., 0., 0.],
         [0., 1., 1., 1., 0., 0., 0., 0., 0.],
         [0., 1., 1., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 1., 1., 0., 0., 0., 0.]]),
 torch.Size([9, 9]))

**visualization**

![visualization of star expansion](https://deephypergraph.readthedocs.io/en/latest/_images/build_structure_graph_from_star_expansion.png)

모든 hyper edge를 연결하는 virtual vertex(blue)을 actual vertex(red) 다음 순번으로 나열한다.
```
v_mask(Tensor) : tensor([ True,  True,  True,  True,  True, False, False, False, False])
```

hyper edge list를 살폈을 때, 
```
[(0, 1, 2), (1, 2, 3), (1, 2), (0, 3, 4)]
```
| hyper edge | actual vertex | virtual vertex |
| :--------: | :-----------: | :------------: |
| (0, 1, 2) | 0, 1, 2 | 5 |
| (1, 2, 3) | 1, 2, 3 | 6 |
| (1, 2) | 1, 2 | 7 |
| (0, 3, 4) | 0, 3, 4 | 8 |

이렇게 순서대로 virtual vertex가 배정되는데, API를 통해 변환된 graph edge list는 다음과 같다. 
모든 virtual vertex와 actual vertex의 조합으로 생각하면 쉬우며 이를 통해 새로운 그래프의 연결관계를 알 수 있게 된다.

```
[(0, 5), (0, 8), (1, 5), (1, 6), (1, 7), (2, 5), (2, 6), (2, 7), (3, 6), (3, 8), (4, 8)]
```

그렇다면 이를 활용하기 위해서는 어떻게 해야할까. 

1. graph의 edge 개수
    조합으로 풀기에는 경우의 수가 정해져 있으므로 virtual vertex를 기준으로 풀어낸 actual vertex의 sum을 구하는 것이 빠르다. 예로, `sum(sum(g.A.to_dense()))//2` 를 들 수 있겠다. 연결된 vertex들의 indices에 1을 할당한 행렬이 $A$ 이므로 그 경우의 수는 위 식으로 알 수 있다.

2. A의 크기
    A를 이용해 학습하므로, A의 형태를 알 필요가 있다. star expansion에서 A는 virtual vertex를 포함한 연결관계를 나타내므로 `v_mask * v_mask` 로 표현할 수 있다.

tensor(22.)

##### Clique Expansion

##### HyperGCN based Expansion

### 실험 결과 및 분석

### 고찰